In [22]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [134]:
len_data_arr = []
results_data_arr_agglo = []
results_data_arr_bunch = []

bunch_hypertune = ['turbomqincrw', 'turbomqincr', 'turbomqw', 'turbomq', 'basicmq']

for root, dirs, files in os.walk('Results_19112020'):
    for file in files:
        #print(file)
        #print(file.split('_')[-1])
        if file.split('_')[-1] == 'len.txt':
            print(file, 'len')
            proj_len_file = open('Results_19112020/' + file)
            proj_name = file.split('_')[0]
            proj_len = proj_len_file.readline().replace('\n','')
            len_data_arr.append([proj_name, proj_len])
            proj_len_file.close()
        elif file.split('_')[-1] == 'results.txt':
            #print(file, 'results')
            proj_result_file = open('Results_19112020/' + file)
            proj_name = file.split('_')[0]
            
            initial_line = 2 - 1
            initial_line_result = 5 - 1
            tmp_arr = []
            
            for i, line in enumerate(proj_result_file):
                
                if i == initial_line:
                    line = line.replace('MoJo', '')
                    line = line.replace('.rsf', '')
                    line = line.replace('(', '')
                    line = line.replace(')', '')
                    line = line.split('_')
                    name = line[1].split('/')[-1]
                    #version = line[0].split('-')[-1]
                    version = line[2]
                    
                    ## If gathers results for bunch
                    if line[3] == 'exhaustive' or line[3] == 'hillclimbing' or line[3] == 'ga':
                        
                        if line[4] in bunch_hypertune:
                            bunch_bool = True
                            method = line[3]
                            calculator = line[4]
                            tmp_arr.append(name)
                            tmp_arr.append(version)
                            tmp_arr.append(method)
                            tmp_arr.append(calculator)

                        initial_line += 5

                    
                    
                    ## Else gathers results for agglomerative
                    else:
                        agglo_bool = True
                        n_cluster = line[3]
                        affinity = line[4]
                        linkage = line[5]
                        #print(line)
                        #print('Version', version)
                        initial_line += 5
                        tmp_arr.append(name)
                        tmp_arr.append(version)
                        tmp_arr.append(n_cluster)
                        tmp_arr.append(affinity)
                        tmp_arr.append(linkage)
                        
                elif i == initial_line_result:
                    n_MoJo = line.split(' ')[-1].split('\n')[0]

                    #print(line)
                    initial_line_result += 5
                    tmp_arr.append(n_MoJo)
                    #tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
                    
                    if agglo_bool == True:
                        results_data_arr_agglo.append(tmp_arr)
                        agglo_bool = False
                    elif bunch_bool == True:
                        results_data_arr_bunch.append(tmp_arr)
                        bunch_bool = False
                    tmp_arr = []
                    
                    
                
                

aws-aws-sdk-java-v2_max_len.txt len
bdew-neiaddons_max_len.txt len
bkromhout-realm-java_max_len.txt len
btraceio-btrace_max_len.txt len
bytedeco-javacpp_max_len.txt len
codecentric-spring-boot-admin_max_len.txt len
codenvy-legacy-che-plugins_max_len.txt len
coobird-thumbnailator_max_len.txt len
cryptomator-cryptomator_max_len.txt len
dropwizard-dropwizard_max_len.txt len
dropwizard-metrics_max_len.txt len
evant-gradle-retrolambda_max_len.txt len
facebook-facebook-android-sdk_max_len.txt len
facebook-facebook-java-business-sdk_max_len.txt len
facebook-fresco_max_len.txt len
facebook-litho_max_len.txt len
facebook-react-native-fbsdk_max_len.txt len
flyway_flyway_max_len.txt len
google-cdep_max_len.txt len
google-dagger_max_len.txt len
google-error-prone_max_len.txt len
google-exoplayer_max_len.txt len
google-gitiles_max_len.txt len
google-openrtb-doubleclick_max_len.txt len
google-openrtb_max_len.txt len
grpc-grpc-java_max_len.txt len
havarunner-havarunner_max_len.txt len
hazelcast-hazel

In [135]:
pd.DataFrame(results_data_arr_agglo)

,0,1,2,3,4,5
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695
...,...,...,...,...,...,...
63073,web3j-web3j,v4.5.2,25,manhattan,average,66
63074,web3j-web3j,v4.5.2,25,manhattan,single,59
63075,web3j-web3j,v4.5.2,25,cosine,complete,205
63076,web3j-web3j,v4.5.2,25,cosine,average,251


In [136]:
pd.DataFrame(results_data_arr_bunch)

,0,1,2,3,4
0,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincrw,102
1,cryptomator-cryptomator,1.5.6,hillclimbing,basicmq,109
2,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincr,120
3,cryptomator-cryptomator,1.5.6,hillclimbing,turbomq,103
4,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqw,116
...,...,...,...,...,...
9706,web3j-web3j,v4.5.2,ga,turbomqincrw,360
9707,web3j-web3j,v4.5.2,ga,basicmq,336
9708,web3j-web3j,v4.5.2,ga,turbomqincr,321
9709,web3j-web3j,v4.5.2,ga,turbomq,348


In [137]:
pd.DataFrame(len_data_arr)

,0,1
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154
...,...,...
72,swagger-api-swagger-core,763
73,testcontainers-testcontainers-java,151
74,uber-NullAway,106
75,voxeolabs-moho,639


In [138]:
agglo_results = pd.DataFrame(results_data_arr_agglo)
agglo_results.columns = ['project_name', 'project_version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results']
agglo_results.head()

,project_name,project_version,cluster_division,affinity,linkage,MoJo_results
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695


In [139]:
bunch_results = pd.DataFrame(results_data_arr_bunch)
bunch_results.columns = ['project_name', 'project_version', 'method', 'calculator', 'MoJo_results']
bunch_results.head()

,project_name,project_version,method,calculator,MoJo_results
0,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincrw,102
1,cryptomator-cryptomator,1.5.6,hillclimbing,basicmq,109
2,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqincr,120
3,cryptomator-cryptomator,1.5.6,hillclimbing,turbomq,103
4,cryptomator-cryptomator,1.5.6,hillclimbing,turbomqw,116


In [140]:
bunch_results['project_name'].unique()

array(['cryptomator-cryptomator', 'dropwizard-dropwizard',
       'dropwizard-metrics', 'evant-gradle-retrolambda',
       'facebook-facebook-android-sdk',
       'facebook-facebook-java-business-sdk', 'facebook-fresco',
       'facebook-litho', 'facebook-react-native-fbsdk', 'google-cdep',
       'google-dagger', 'google-error-prone', 'google-exoplayer',
       'google-gitiles', 'google-openrtb-doubleclick', 'google-openrtb',
       'grpc-grpc-java', 'havarunner-havarunner', 'immutables-immutables',
       'ionic-team-capacitor', 'iSoron-uhabits', 'jankotek-mapdb',
       'java-native-accessjna', 'javafunk-funk', 'javaparser-javaparser',
       'jboss-switchyard-release', 'jenkinsci-jenkins', 'jhy-jsoup',
       'joelittlejohn-jsonschema2pojo', 'kijiproject-kiji-bento',
       'knowm-XChange', 'lettuce-io-lettuce-core', 'mockito-mockito',
       'mrniko-netty-socketio', 'mybatis-mybatis-3', 'naver-pinpoint',
       'netty-netty', 'oblac-jodd', 'openengsb-openengsb',
       'oracle-oci

In [141]:
proj_len_df = pd.DataFrame(len_data_arr)
proj_len_df.columns = ['project_name', 'len']
proj_len_df.head()

,project_name,len
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154


In [142]:
## Remove flyway error
agglo_results = agglo_results[agglo_results['project_name'] != 'flyway']
agglo_results = agglo_results[agglo_results['project_name'] != 'mrniko-netty-socketio']



In [143]:
#agglo_results.to_csv('agglo_results_21102020.csv', index=False)
#bunch_results.to_csv('bunch_results_21102020.csv', index=False)

In [144]:
proj_len_df.to_csv('proj_len.csv', index=False)

In [145]:
main_df_agglo = agglo_results.merge(proj_len_df, on='project_name', how='left')
main_df_agglo['len'] = main_df_agglo['len'].astype('int32')
main_df_agglo['cluster_division'] = main_df_agglo['cluster_division'].astype('int32')
main_df_agglo['MoJo_results'] = main_df_agglo['MoJo_results'].astype('int32')
main_df_agglo['num_clusters'] = main_df_agglo['len'] // main_df_agglo['cluster_division']
main_df_agglo['accuracy'] = 1-  (main_df_agglo['MoJo_results'] / main_df_agglo['len'])
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l2']
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l1']
main_df_agglo = main_df_agglo[main_df_agglo['accuracy'] != 1]
main_df_agglo = main_df_agglo.groupby(['project_name', 'project_version']).apply(lambda x: x.nlargest(1, "accuracy")).reset_index(drop=True)
main_df_agglo.head(20)

,project_name,project_version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy
0,aws-aws-sdk-java-v2,2.9.24,25,manhattan,single,280,2168,86,0.870849
1,aws-aws-sdk-java-v2,2.9.25,25,cosine,single,276,2168,86,0.872694
2,aws-aws-sdk-java-v2,2.9.26,25,cosine,single,276,2168,86,0.872694
3,aws-aws-sdk-java-v2,2.9.3,25,manhattan,single,282,2168,86,0.869926
4,aws-aws-sdk-java-v2,2.9.4,25,manhattan,single,282,2168,86,0.869926
5,aws-aws-sdk-java-v2,2.9.5,25,manhattan,single,282,2168,86,0.869926
6,aws-aws-sdk-java-v2,2.9.6,25,manhattan,single,282,2168,86,0.869926
7,aws-aws-sdk-java-v2,2.9.7,25,manhattan,single,282,2168,86,0.869926
8,aws-aws-sdk-java-v2,2.9.8,25,manhattan,single,282,2168,86,0.869926
9,aws-aws-sdk-java-v2,2.9.9,25,manhattan,single,281,2168,86,0.870387


In [146]:
main_df_bunch = bunch_results.merge(proj_len_df, on='project_name', how='left')
main_df_bunch['len'] = main_df_bunch['len'].astype('int32')
main_df_bunch['MoJo_results'] = main_df_bunch['MoJo_results'].astype('int32')
main_df_bunch['accuracy'] = 1-  (main_df_bunch['MoJo_results'] / main_df_bunch['len'])
main_df_bunch = main_df_bunch.groupby(['project_name', 'project_version']).apply(lambda x: x.nlargest(1, "accuracy")).reset_index(drop=True)
main_df_bunch.head(10)

,project_name,project_version,method,calculator,MoJo_results,len,accuracy
0,cryptomator-cryptomator,1.5.0-beta3,exhaustive,turbomqincr,172,222,0.225225
1,cryptomator-cryptomator,1.5.0-rc1,exhaustive,turbomq,163,222,0.265766
2,cryptomator-cryptomator,1.5.0-rc2,exhaustive,turbomqincr,162,222,0.270270
3,cryptomator-cryptomator,1.5.0-rc3,exhaustive,turbomq,166,222,0.252252
4,cryptomator-cryptomator,1.5.1,ga,turbomqw,165,222,0.256757
5,cryptomator-cryptomator,1.5.2,exhaustive,turbomq,98,222,0.558559
6,cryptomator-cryptomator,1.5.3,exhaustive,basicmq,93,222,0.581081
7,cryptomator-cryptomator,1.5.4,hillclimbing,turbomqw,95,222,0.572072
8,cryptomator-cryptomator,1.5.5,exhaustive,turbomqw,96,222,0.567568
9,cryptomator-cryptomator,1.5.6,ga,turbomqincr,100,222,0.549550


In [147]:
agglo_minimal = main_df_agglo[['project_name', 'project_version', 'accuracy']]
agglo_minimal.columns = ['project_name', 'project_version', 'agglo_accuracy']
bunch_minimal = main_df_bunch[['project_name', 'project_version', 'accuracy']]
bunch_minimal.columns = ['project_name', 'project_version', 'bunch_accuracy']
combine_df = agglo_minimal.merge(bunch_minimal, on=['project_name', 'project_version'])

In [148]:
combine_df.sort_values(['agglo_accuracy'], ascending=False)

,project_name,project_version,agglo_accuracy,bunch_accuracy
50,facebook-facebook-java-business-sdk,v3.2.7,0.986577,0.479866
451,redisson-redisson,redisson-parent-3.11.6,0.981442,0.289079
362,oracle-oci-java-sdk,v1.7.0,0.976621,0.423592
365,oracle-oci-java-sdk,v1.8.2,0.973220,0.527736
364,oracle-oci-java-sdk,v1.8.1,0.973220,0.288842
...,...,...,...,...
557,square-okhttp,parent-4.6.0,0.451220,0.426829
561,square-okhttp,parent-4.8.0,0.445122,0.439024
559,square-okhttp,parent-4.7.1,0.445122,0.469512
558,square-okhttp,parent-4.7.0,0.445122,0.475610


In [149]:
conditions = [
    (combine_df['agglo_accuracy'] > combine_df['bunch_accuracy']),
    (combine_df['bunch_accuracy'] >= combine_df['agglo_accuracy'])
]

values = ['agglo', 'bunch']

combine_df['algo_choice'] = np.select(conditions, values)

In [150]:
combine_df.sort_values(['agglo_accuracy'], ascending=False).head()

,project_name,project_version,agglo_accuracy,bunch_accuracy,algo_choice
50,facebook-facebook-java-business-sdk,v3.2.7,0.986577,0.479866,agglo
451,redisson-redisson,redisson-parent-3.11.6,0.981442,0.289079,agglo
362,oracle-oci-java-sdk,v1.7.0,0.976621,0.423592,agglo
365,oracle-oci-java-sdk,v1.8.2,0.973220,0.527736,agglo
364,oracle-oci-java-sdk,v1.8.1,0.973220,0.288842,agglo


In [151]:
combine_df.to_csv('combine_algo_choice_19112020.csv', index=False)

In [152]:
combine_df.groupby('algo_choice').count()

,project_name,project_version,agglo_accuracy,bunch_accuracy
algo_choice,,,,
agglo,608,608,608,608
bunch,14,14,14,14


In [153]:
ck_max = pd.read_csv('ck_max_194.csv')
ck_std = pd.read_csv('ck_std_194.csv')
ck_mean = pd.read_csv('ck_mean_194.csv')
ck_sum = pd.read_csv('ck_sum_194.csv')

In [154]:
ck_max_column = ['project_name', 'project_version']
ck_std_column = ['project_name', 'project_version']
ck_mean_column = ['project_name', 'project_version']
ck_sum_column = ['project_name', 'project_version']

excluded_columns = ['project_name', 'project_version']



for element in ck_max.columns:
    #print(element)
    if element not in excluded_columns:
        ck_max_column.append('feature_' + element + '_max')
   
        
        
for element in ck_std.columns:
    #print(element)
    if element not in excluded_columns:
        ck_std_column.append('feature_' + element + '_std')
        
for element in ck_mean.columns:
    #print(element)
    if element not in excluded_columns:
        ck_mean_column.append('feature_' + element + '_mean')
        
for element in ck_sum.columns:
    #print(element)
    if element not in excluded_columns:
        ck_sum_column.append('feature_' + element + '_sum')
    
        
ck_max.columns = ck_max_column
ck_std.columns = ck_std_column
ck_sum.columns = ck_sum_column
ck_mean.columns = ck_mean_column


ck_max.head()

,project_name,project_version,feature_cbo_max,feature_wmc_max,feature_dit_max,feature_rfc_max,feature_lcom_max,feature_totalMethods_max,feature_staticMethods_max,feature_publicMethods_max,...,feature_numbersQty_max,feature_assignmentsQty_max,feature_mathOperationsQty_max,feature_variablesQty_max,feature_maxNestedBlocks_max,feature_anonymousClassesQty_max,feature_subClassesQty_max,feature_lambdasQty_max,feature_uniqueWordsQty_max,feature_modifiers_max
0,JodaOrg-joda-time,v2.10.6,79.0,203.0,17.0,200.0,6811.0,131.0,58.0,123.0,...,2067.0,353.0,195.0,297.0,6.0,21.0,13.0,0.0,573.0,1025.0
1,JodaOrg-joda-time,v2.2,78.0,203.0,17.0,194.0,6578.0,129.0,71.0,121.0,...,2067.0,353.0,195.0,292.0,6.0,21.0,13.0,0.0,553.0,1025.0
2,JodaOrg-joda-time,v2.3,78.0,203.0,17.0,194.0,6811.0,129.0,58.0,121.0,...,2067.0,353.0,195.0,292.0,6.0,21.0,13.0,0.0,554.0,1025.0
3,JodaOrg-joda-time,v2.4,78.0,203.0,17.0,195.0,6811.0,131.0,58.0,123.0,...,2067.0,353.0,195.0,297.0,6.0,21.0,13.0,0.0,554.0,1025.0
4,JodaOrg-joda-time,v2.5,78.0,203.0,17.0,195.0,6811.0,131.0,58.0,123.0,...,2067.0,353.0,195.0,297.0,6.0,21.0,13.0,0.0,561.0,1025.0


In [155]:
main_df_agglo = main_df_agglo.merge(ck_max, how='left', on=['project_name', 'project_version'])
main_df_agglo = main_df_agglo.merge(ck_mean, how='left', on=['project_name', 'project_version'])
main_df_agglo = main_df_agglo.merge(ck_std, how='left', on=['project_name', 'project_version'])
main_df_agglo = main_df_agglo.merge(ck_sum, how='left', on=['project_name', 'project_version'])

main_df_agglo.head()

,project_name,project_version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,feature_cbo_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,aws-aws-sdk-java-v2,2.9.24,25,manhattan,single,280,2168,86,0.870849,138.0,...,5625.0,16197.0,2432.0,14775.0,1243.0,330.0,835.0,1878.0,124807.0,128965.0
1,aws-aws-sdk-java-v2,2.9.25,25,cosine,single,276,2168,86,0.872694,138.0,...,5631.0,16224.0,2433.0,14796.0,1245.0,333.0,837.0,1889.0,125059.0,128952.0
2,aws-aws-sdk-java-v2,2.9.26,25,cosine,single,276,2168,86,0.872694,138.0,...,5631.0,16224.0,2433.0,14796.0,1245.0,333.0,837.0,1889.0,125059.0,128952.0
3,aws-aws-sdk-java-v2,2.9.3,25,manhattan,single,282,2168,86,0.869926,138.0,...,5559.0,16029.0,2396.0,14620.0,1236.0,330.0,826.0,1849.0,123843.0,127845.0
4,aws-aws-sdk-java-v2,2.9.4,25,manhattan,single,282,2168,86,0.869926,138.0,...,5559.0,16029.0,2396.0,14620.0,1236.0,330.0,826.0,1849.0,123843.0,127845.0


In [156]:
main_df_bunch = main_df_bunch.merge(ck_max, how='left', on=['project_name', 'project_version'])
main_df_bunch = main_df_bunch.merge(ck_mean, how='left', on=['project_name', 'project_version'])
main_df_bunch = main_df_bunch.merge(ck_std, how='left', on=['project_name', 'project_version'])
main_df_bunch = main_df_bunch.merge(ck_sum, how='left', on=['project_name', 'project_version'])

main_df_bunch.head()

,project_name,project_version,method,calculator,MoJo_results,len,accuracy,feature_cbo_max,feature_wmc_max,feature_dit_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,cryptomator-cryptomator,1.5.0-beta3,exhaustive,turbomqincr,172,222,0.225225,41.0,49.0,6.0,...,268.0,1132.0,96.0,1158.0,100.0,7.0,44.0,111.0,6898.0,17591.0
1,cryptomator-cryptomator,1.5.0-rc1,exhaustive,turbomq,163,222,0.265766,40.0,53.0,7.0,...,268.0,1157.0,95.0,1184.0,102.0,7.0,46.0,112.0,7063.0,18618.0
2,cryptomator-cryptomator,1.5.0-rc2,exhaustive,turbomqincr,162,222,0.270270,40.0,53.0,7.0,...,268.0,1158.0,95.0,1185.0,102.0,7.0,46.0,113.0,7073.0,18618.0
3,cryptomator-cryptomator,1.5.0-rc3,exhaustive,turbomq,166,222,0.252252,40.0,53.0,7.0,...,268.0,1158.0,95.0,1185.0,102.0,7.0,46.0,113.0,7075.0,18618.0
4,cryptomator-cryptomator,1.5.1,ga,turbomqw,165,222,0.256757,40.0,58.0,7.0,...,268.0,1164.0,97.0,1191.0,101.0,7.0,46.0,114.0,7136.0,18619.0


In [157]:
main_df_agglo.to_csv('agglo_results_19112020.csv', index=False)
main_df_bunch.to_csv('bunch_results_19112020.csv', index=False)

In [158]:
main_df_agglo = pd.read_csv('agglo_results_19112020.csv')
main_df_bunch = pd.read_csv('bunch_results_19112020.csv')

In [131]:
### Following is to change to Dr Aldeida's format
dr_main_df_agglo = main_df_agglo
dr_main_df_agglo['Instances'] = dr_main_df_agglo['project_name'] + dr_main_df_agglo['project_version']
dr_main_df_agglo[''] = dr_main_df_agglo['cluster_division'].astype(str) + '_' + dr_main_df_agglo['linkage'] + '_' + dr_main_df_agglo['affinity']

dr_main_df_agglo.drop('project_name', axis=1, inplace=True)
dr_main_df_agglo.drop('project_version', axis=1, inplace=True)
dr_main_df_agglo.drop('cluster_division', axis=1, inplace=True)
dr_main_df_agglo.drop('affinity', axis=1, inplace=True)
dr_main_df_agglo.drop('linkage', axis=1, inplace=True)
dr_main_df_agglo.drop('MoJo_results', axis=1, inplace=True)
dr_main_df_agglo.drop('len', axis=1, inplace=True)
dr_main_df_agglo.drop('num_clusters', axis=1, inplace=True)
dr_main_df_agglo.drop('accuracy', axis=1, inplace=True)

dr_main_df_agglo.head()

print(dr_main_df_agglo.columns)


Index(['cbo_max', 'wmc_max', 'dit_max', 'rfc_max', 'lcom_max',
       'totalMethods_max', 'staticMethods_max', 'publicMethods_max',
       'privateMethods_max', 'protectedMethods_max',
       ...
       'mathOperationsQty_sum', 'variablesQty_sum', 'maxNestedBlocks_sum',
       'anonymousClassesQty_sum', 'subClassesQty_sum', 'lambdasQty_sum',
       'uniqueWordsQty_sum', 'modifiers_sum', 'project', 'config'],
      dtype='object', length=162)


In [132]:
### Following is to change to Dr Aldeida's format
dr_main_df_bunch = main_df_bunch
dr_main_df_bunch['project'] = dr_main_df_bunch['project_name'] + dr_main_df_bunch['project_version']
dr_main_df_bunch['config'] = dr_main_df_bunch['method']

dr_main_df_bunch.drop('project_name', axis=1, inplace=True)
dr_main_df_bunch.drop('project_version', axis=1, inplace=True)
dr_main_df_bunch.drop('method', axis=1, inplace=True)
dr_main_df_bunch.drop('MoJo_results', axis=1, inplace=True)
dr_main_df_bunch.drop('len', axis=1, inplace=True)
dr_main_df_bunch.drop('accuracy', axis=1, inplace=True)

dr_main_df_bunch.head()

print(dr_main_df_bunch.columns)


Index(['calculator', 'cbo_max', 'wmc_max', 'dit_max', 'rfc_max', 'lcom_max',
       'totalMethods_max', 'staticMethods_max', 'publicMethods_max',
       'privateMethods_max',
       ...
       'mathOperationsQty_sum', 'variablesQty_sum', 'maxNestedBlocks_sum',
       'anonymousClassesQty_sum', 'subClassesQty_sum', 'lambdasQty_sum',
       'uniqueWordsQty_sum', 'modifiers_sum', 'project', 'config'],
      dtype='object', length=163)


In [133]:
dr_main_df_agglo.to_csv('draldeida_agglo_results_19112020.csv', index=False)
dr_main_df_bunch.to_csv('draldeida_bunch_results_19112020.csv', index=False)

In [28]:
main_df_agglo.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,cbo_max,...,numbersQty_sum,assignmentsQty_sum,mathOperationsQty_sum,variablesQty_sum,maxNestedBlocks_sum,anonymousClassesQty_sum,subClassesQty_sum,lambdasQty_sum,uniqueWordsQty_sum,modifiers_sum
0,aws-aws-sdk-java-v2,2.9.9,25,manhattan,single,281,2168,86,0.870387,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
1,aws-aws-sdk-java-v2,2.9.8,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
2,aws-aws-sdk-java-v2,2.9.7,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
3,aws-aws-sdk-java-v2,2.9.6,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
4,aws-aws-sdk-java-v2,2.9.5,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0


In [90]:
g = main_df_agglo.groupby('project_name')

first = g['project_name', 'version'].head(1)
first.columns = ['project_name', 'firstRelease']

last = g['project_name', 'version'].tail(1)
last.columns = ['project_name', 'lastRelease']


project_first_last = first.merge(last, on='project_name')
#project_first_last = (pd.concat([g.head(1), g.tail(1)]).reset_index(drop=True).sort_values('project_name'))
pd.set_option('display.max_rows', 999)
pd.set_option('display.width', None)
project_first_last.head(71)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,project_name,firstRelease,lastRelease
0,aws-aws-sdk-java-v2,2.9.9,2.9.24
1,bdew-neiaddons,v1.9.4,v1.8.0
2,bkromhout-realm-java,v0.89.0,v0.87.1
3,btraceio-btrace,v1.3.9,v1.3.4
4,bytedeco-javacpp,1.5.3,1.4
5,codecentric-spring-boot-admin,1.5.7,1.4.5
6,codenvy-legacy-che-plugins,3.9.5,3.13.4.4
7,coobird-thumbnailator,0.4.9,0.4.10
8,cryptomator-cryptomator,1.5.6,1.5.0-beta3
9,dropwizard-dropwizard,v2.0.9,v2.0.11


In [48]:
len(project_first_last)

71

In [50]:
g_bunch = main_df_bunch.groupby('project_name')

first_bunch = g_bunch['project_name', 'version'].head(1)
first_bunch.columns = ['project_name', 'startRelease']

last_bunch = g_bunch['project_name', 'version'].tail(1)
last_bunch.columns = ['project_name', 'endRelease']


project_first_last_bunch = first_bunch.merge(last_bunch, on='project_name')
#project_first_last = (pd.concat([g.head(1), g.tail(1)]).reset_index(drop=True).sort_values('project_name'))
project_first_last_bunch.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,project_name,startRelease,endRelease
0,aws-aws-sdk-java-v2,2.9.9,2.9.24
1,bdew-neiaddons,v1.9.4,v1.7.1
2,bkromhout-realm-java,v0.89.0,v0.87.1
3,btraceio-btrace,v1.3.9,v1.3.4
4,bytedeco-javacpp,1.5.3,1.4
5,codecentric-spring-boot-admin,1.5.7,1.4.5
6,codenvy-legacy-che-plugins,3.9.5,3.13.4.4
7,coobird-thumbnailator,0.4.9,0.4.10
8,cryptomator-cryptomator,1.5.6,1.5.0-beta3
9,dropwizard-dropwizard,v2.0.9,v2.0.11


In [51]:
len(project_first_last_bunch)

74

In [52]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]
#print(Diff(li1, li2))


print(Diff(project_first_last_bunch['project_name'], project_first_last['project_name']))

['hibernate-hibernate-orm', 'realm-realm-java', 'mrniko-netty-socketio']


In [53]:
project_first_last.to_csv('projectStartEndReleaseAgglo.csv', index=False)
project_first_last_bunch.to_csv('projectStartEndReleaseBunch.csv', index=False)

In [84]:
### Visualization
visual_bunch_df = main_df_bunch.copy()
visual_bunch_df.columns = ['project_name', 'version', 'method', 'MoJo_results', 'num_classes', 'MoJoFM']
visual_bunch_df.drop(columns=['MoJo_results'], inplace=True)
visual_bunch_df['MoJoFM'] = visual_bunch_df['MoJoFM'] * 100

visual_bunch_df.sort_values(['MoJoFM'], ascending=False).head(10)

,project_name,version,method,num_classes,MoJoFM
661,google-openrtb,1.5.9,ga,44,88.636364
666,google-openrtb,1.5.7,hillclimbing,44,86.363636
948,javafunk-funk,funk-0.1.4,hillclimbing,268,85.820896
951,javafunk-funk,funk-0.1.3,hillclimbing,268,83.208955
676,google-openrtb,1.5.4,ga,44,81.818182
686,google-openrtb,1.5.12,exhaustive,44,81.818182
672,google-openrtb,1.5.5,hillclimbing,44,81.818182
563,google-dagger,dagger-2.3,exhaustive,1052,80.988593
498,facebook-react-native-fbsdk,0.7.0,hillclimbing,21,80.952381
496,facebook-react-native-fbsdk,0.8.0,ga,21,80.952381


In [83]:
### Visualization
visual_agglo_df = main_df_agglo.copy()
visual_agglo_df.columns = ['project_name', 'version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results', 'num_classes', 'num_clusters', 'MoJoFM']
visual_agglo_df.drop(columns=['MoJo_results'], inplace=True)
visual_agglo_df['MoJoFM'] = visual_agglo_df['MoJoFM'] * 100

visual_agglo_df.sort_values(['MoJoFM'], ascending=False).head(10)

,project_name,version,cluster_division,affinity,linkage,num_classes,num_clusters,MoJoFM
11762,facebook-facebook-java-business-sdk,v3.2.7,25,euclidean,single,596,23,98.657718
45221,redisson-redisson,redisson-parent-3.11.6,25,manhattan,single,1401,56,98.144183
38894,oracle-oci-java-sdk,v1.7.0,25,cosine,single,4705,188,97.662062
38612,oracle-oci-java-sdk,v1.8.2,25,euclidean,single,4705,188,97.321998
38702,oracle-oci-java-sdk,v1.8.1,25,euclidean,single,4705,188,97.321998
38792,oracle-oci-java-sdk,v1.8.0,25,euclidean,single,4705,188,97.258236
38522,oracle-oci-java-sdk,v1.9.0,25,euclidean,single,4705,188,97.258236
38432,oracle-oci-java-sdk,v1.9.1,25,euclidean,single,4705,188,97.236982
11672,facebook-facebook-java-business-sdk,v3.2.8,25,euclidean,single,596,23,96.979866
11582,facebook-facebook-java-business-sdk,v3.2.9,25,euclidean,single,596,23,96.812081
